In [31]:
import os
import tensorflow as tf
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gym
from gym import spaces
from ray import tune
import sys
# sys.path.append('/Users/sakhtar/projects/DRL_PurdueAg/')
import rom
import utils
import ray.rllib.agents.ppo as ppo

In [ ]:
# df = pd.read_csv('data/IO_data_SupervisoryController.csv', skiprows=3)

In [33]:
config = ppo.DEFAULT_CONFIG.copy()

In [54]:
config['evaluation_num_episodes']

-1

# DRL Outline

1. Discrete points of observation and actions
2. Interval b/w 2 consecutive observation - Agent observes at time, t. For this iteration the interval is set at 1 second
3. Each episode comprises of a 300 second time
4. Action will be taken by the agent after 1 second

## Action

1. Medium pressure ($P_{MP}$) and High Pressure ($P_{HP}$) levels
2. Operating mode of the actuators: $mode_{op} \in \{1,2,3\} $
3. $mode^1_{op} = P_{MP} - P_{LP}$
4. $mode^2_{op} = P_{HP} - P_{MP}$
5. $mode^3_{op} = P_{HP} - P_{LP}$
## State/Observation

1. Throttling valve pressures of each actuator:  $\Delta P_{i}$, where $i \in \{ bulk, fan, vac, alt\}$  
2. Fluid flow rates across each actuator:  $Q_{i}$, where $i \in \{ bulk, fan, vac, alt\}$  
3. Deviation between commanded vs actual rotation speed of each actuator:  $\Delta \omega_{i}$, where $i \in \{ bulk, fan, vac, alt\}$

## Environment variables

For this problem, the environment variable is the commanded speed of the actuator. The commanded speeds determine the operating pressure required by the actuators to maintain that speed

## Transition - Step function

1. Given state, $s_t$, action, $a_t$, and environment variable, $e_t$, predict, $s_{t+1}$
2. The ROM governs the transition/step function
3. State $s_{t+1}$ should only depend on $a_t$ and $s_t$ and should be independent of the history -> Ensures that the environment is a Markov decision process

Should the environment variable $e_t$ determine the bounds for action variable? How can that be enforced?

## Reward function

Reward function in this case has five components to it, all of which need to be minimized:

1. $W_{loss} = (P_{available} - \Delta P_{i})*Q_{i}$
2. $ \omega_{deviation} = (\omega^{t}_{target, i} - \omega^{t}_{actual, i})^2$
3. $ pHP_{deviation} =  pHP^{t+1} - pHP^{t}$
4. $ pMP_{deviation} =  pMP^{t+1} - pMP^{t}$
5. $ mode_{i, deviation} = mode^{t+1}_{i} - mode^{t}_{i} $

In [15]:
d = spaces.MultiDiscrete([3, 3, 3, 3])
e = spaces.Box(low=1, high=300, shape=(2,), dtype=np.float32)

e.sample(), d.sample()

(array([92.36637,  6.69775], dtype=float32), array([2, 1, 1, 1]))

In [16]:
max_operating_pressure = 300  
action_space = spaces.Dict({
            'actuator_mode': spaces.MultiDiscrete([3, 3, 3, 3]),  # Example discrete action with 4 options corresponding to 4 modes of the actuators
            'pHP': spaces.Box(low=1, high=max_operating_pressure, shape=(1,), dtype=np.float32),
              'ratio': spaces.Box(low=0, high = 1, shape=(1,), dtype = np.float32),# Example continuous action with shape (2,) for MP and HP
        })
action_space.sample()

OrderedDict([('actuator_mode', array([1, 2, 1, 0])),
             ('pHP', array([6.439041], dtype=float32)),
             ('ratio', array([0.2532038], dtype=float32))])

In [17]:
high_pressure = np.random.uniform(low=1, high=300)
medium_pressure = np.random.uniform()*high_pressure
np.array([[high_pressure],[medium_pressure]])

array([[219.06484005],
       [ 32.49306261]])

In [18]:
num_actuators = 4
max_delta_rpm = 4000               #rpm
max_operating_flowrate = 50        #lpm
max_operating_pressure = 300   #bar
min_rail_pressure = 20
max_rpm = np.array([4998,4950,3915.66,3000]) # max rpm for each actuator (bulk, vac, alt, fert)
obs_low = np.zeros((12,1))
obs_high = np.array([[max_operating_pressure] for _ in range(num_actuators)] + 
                            [[max_operating_flowrate] for _ in range(num_actuators)] +
                            [[max_delta_rpm] for _ in range(num_actuators)])
observation_space = spaces.Box(low=obs_low, high=obs_high, dtype=np.float32)

/opt/anaconda3/envs/PurdueAg/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [47]:
comp_time = 3670

In [49]:
(comp_time%60)

10

In [19]:
observation_space.sample()

array([[ 256.00385 ],
       [  24.81849 ],
       [ 282.6233  ],
       [ 232.44662 ],
       [  28.05222 ],
       [  30.68227 ],
       [   9.982396],
       [   9.73631 ],
       [ 877.2622  ],
       [1765.1543  ],
       [ 810.432   ],
       [1461.726   ]], dtype=float32)

In [35]:
T, data = utils.load_data('../data/data.h5')
Bulk_P = data['Bulk_P']
Alt_P = data['Alt_P']
Vac_P = data['Vac_P']
Fert_P = data['Fert_P']
Bulk_Q = data['Bulk_Q']
Alt_Q = data['Alt_Q']
Vac_Q = data['Vac_Q']
Fert_Q = data['Fert_Q']
Bulk_rpm_delta = data['Bulk_rpm_delta']
Alt_rpm_delta = data['Alt_rpm_delta']
Vac_rpm_delta = data['Vac_rpm_delta']
Fert_rpm_delta = data['Fert_rpm_delta']
k=np.random.randint(0,len(Bulk_P))
init_obs = {'continuous': np.array([Bulk_P[k,k], Alt_P[k, k], Vac_P[k, k], Fert_P[k, k],
                                Bulk_Q[k, k], Alt_Q[k, k], Vac_Q[k, k], Fert_Q[k, k],
                                Bulk_rpm_delta[k, k], Alt_rpm_delta[k, k], 
                                Vac_rpm_delta[k, k], Fert_rpm_delta[k, k]]).reshape(12,1)}

In [43]:
Bulk_P.shape[1]

1501

In [22]:
np.array([[Bulk_P[5,5]]])

array([[73.93665592]])

In [23]:
obs_low = np.zeros((12,1))
num_actuators = 4
max_delta_rpm = 4000               #rpm
max_operating_flowrate = 50        #lpm
max_operating_pressure = 300   #bar
min_rail_pressure = 20
max_rpm = np.array([4998,4950,3915.66,3000]) 
actuator_modes = np.array([1,2,0,1])

action = {'discrete': actuator_modes.reshape(4,1),
                'continuous': np.array([[high_pressure],[medium_pressure]])}
commanded_RPM = np.random.uniform(low=0, high=max_delta_rpm)*np.ones(num_actuators)
cmd_rpm = {'continuous': commanded_RPM.reshape(4,1)}

In [26]:
d_x, d_f = 22, 12
sf = utils.load_scale_factors('../model/scale_factors.h5')
load_model_path = '../model/rom.h5'
headers_in = {'in':['pHP', 'pMP', 'Bulk_mode', 'Alt_mode', 'Vac_mode', 'Fert_mode']}
headers_env = {'env':['Bulk_rpm_cmd', 'Alt_rpm_cmd', 'Vac_rpm_cmd', 'Fert_rpm_cmd']}
headers_out = {'out':['Bulk_P', 'Alt_P', 'Vac_P', 'Fert_P', 
            'Bulk_Q', 'Alt_Q', 'Vac_Q', 'Fert_Q',
            'Bulk_rpm_delta', 'Alt_rpm_delta', 'Vac_rpm_delta', 'Fert_rpm_delta']}


# headers = headers_in + headers_env + headers_out

headers = {'in':['pHP', 'pMP', 'Bulk_mode', 'Alt_mode', 'Vac_mode', 'Fert_mode'],
           'env':['Bulk_rpm_cmd', 'Alt_rpm_cmd', 'Vac_rpm_cmd', 'Fert_rpm_cmd'],
           "out":['Bulk_P', 'Alt_P', 'Vac_P', 'Fert_P', 
            'Bulk_Q', 'Alt_Q', 'Vac_Q', 'Fert_Q',
            'Bulk_rpm_delta', 'Alt_rpm_delta', 'Vac_rpm_delta', 'Fert_rpm_delta']}

model = rom.ROM(d_x, d_f, 
                scale_factors=sf, 
                model_path=load_model_path,
               headers=headers)

In [27]:
sf

{'Alt_P': [-0.5000960000000308, 42.02517049306905, 'minmax'],
 'Alt_Q': [0.0, 35.31594534653484, 'minmax'],
 'Alt_mode': [1.0, 3.1, 'minmax'],
 'Alt_rpm_cmd': [-3.8045694207827937e-13, 5197.500000000014, 'minmax'],
 'Alt_rpm_delta': [1.254804365382213e-09, 5197.489499999954, 'minmax'],
 'Bulk_P': [-0.4865862460395955, 119.22507386675787, 'minmax'],
 'Bulk_Q': [-1.0583150504638109e-05, 37.78610102420679, 'minmax'],
 'Bulk_mode': [1.0, 3.1, 'minmax'],
 'Bulk_rpm_cmd': [3000.0, 5097.900000000029, 'minmax'],
 'Bulk_rpm_delta': [2.500356680027268e-07, 5247.907313601388, 'minmax'],
 'Fert_P': [-0.5092473715349765, 102.41206932402267, 'minmax'],
 'Fert_Q': [-1.7178899999999976e-08, 8.96601157016596, 'minmax'],
 'Fert_mode': [1.0, 3.1, 'minmax'],
 'Fert_rpm_cmd': [-6.601935719779907e-13, 3150.000000000009, 'minmax'],
 'Fert_rpm_delta': [-1.0289640481132776e-12, 3150.0000000000086, 'minmax'],
 'Vac_P': [-0.5137579856435713, 220.80298522271073, 'minmax'],
 'Vac_Q': [-0.00021212424999670306, 29.0

In [ ]:
data['Alt_rpm_cmd'][0]

In [ ]:
commanded_RPM = np.array([data['Alt_rpm_cmd'][0],data['Vac_rpm_cmd'][0],
                 data['Fert_rpm_cmd'][0], data['Bulk_rpm_cmd'][0] ]).T

In [ ]:
# commanded_RPM = np.random.uniform(low=0, high=max_delta_rpm)*np.ones(num_actuators)
# cmd_rpm = {'continuous': commanded_RPM.reshape(4,1)}
cmd_rpm = {'continuous':commanded_RPM}

In [ ]:
action = action_space.sample()

actuator_modes = action['actuator_mode']+1
pHP = action['pHP']
pMP = pHP*action['ratio']
action_input = {'discrete': actuator_modes.reshape(4,1),
                'continuous': np.array([pHP,pMP])}

In [ ]:
plt.plot(np.arange(len(commanded_RPM)), commanded_RPM[:,0])
plt.plot(np.arange(len(commanded_RPM)), commanded_RPM[:,1])
plt.plot(np.arange(len(commanded_RPM)), commanded_RPM[:,2])
plt.plot(np.arange(len(commanded_RPM)), commanded_RPM[:,3])

In [ ]:
df = pd.DataFrame(data=commanded_RPM)
df.columns = ['Vac','Fert','Alt','Bulk']
# df['Vac'].apply(lambda x: int(x)).value_counts()

In [ ]:
time = 1500
time_step = 0.2
obs_t = init_obs
obs = [init_obs['continuous'].flatten()]
for t in range(time):
    # action = action_space.sample()
    # actuator_modes = action['actuator_mode']+1
    # pHP = action['pHP']
    # pMP = pHP*action['ratio']
    # action_input = {'discrete': actuator_modes.reshape(4,1),
    #                 'continuous': np.array([pHP,pMP])}
    cmd_rpm = {'continuous':np.array([[c] for c in commanded_RPM[t]])}
    obs_t = model.call_ROM(act = action_input, cmd = cmd_rpm, obs=obs_t)
    obs.append(obs_t['continuous'].flatten())

In [ ]:
# Check if environment works

from controller_env import OptimControllerEnv

env = OptimControllerEnv()
obs = env.reset()

while True:
    action = env.action_space.sample()
    exogenous_variable = env.commanded_RPM
    obs, r, done, _ = env.step(action=action)
    if r > -3000:
        print(f'Reward : {r}, Action : {action}')
        print(obs.flatten())
    
    if done:
        break

In [ ]:
df.plot(x = 'time', y=['Bulk_P','Alt_P','Fert_P','Vac_P',
                       'Bulk_Q', 'Alt_Q', 'Vac_Q','Fert_Q'])

In [ ]:
# np.concatenate((action['continuous'], 
# 							action['discrete'],
# 							cmd_rpm['continuous'],
# 							init_obs['continuous']), axis=0)
action['continuous'].ndim

In [ ]:
class OptimControllerEnv(gym.Env):
    def __init__(self):
        obs_low = np.zeros((12,1))
        self.num_actuators = 4
        self.max_delta_rpm = 10000               #rpm
        self.max_operating_flowrate = 500        #lpm
        self.max_operating_pressure = 3000   #bar
        self.min_rail_pressure = 20
        # self.max_rpm = np.array([4998,4950,3915.66,3000]) # max rpm for each actuator (bulk, vac, alt, fert)
        self.max_rpm = np.array([49980,49500,39150.66,30000])
        d_x, d_f = 22, 12
        sf = utils.load_scale_factors('model/scale_factors.h5')
        load_model_path = 'model/rom.h5'
        headers_in = ['pHP', 'pMP', 'Bulk_mode', 'Alt_mode', 'Vac_mode', 'Fert_mode']
        headers_env = ['Bulk_rpm_cmd', 'Alt_rpm_cmd', 'Vac_rpm_cmd', 'Fert_rpm_cmd']
        headers_out = ['Bulk_P', 'Alt_P', 'Vac_P', 'Fert_P', 
                    'Bulk_Q', 'Alt_Q', 'Vac_Q', 'Fert_Q',
                    'Bulk_rpm_delta', 'Alt_rpm_delta', 'Vac_rpm_delta', 'Fert_rpm_delta']

        headers = headers_in + headers_env + headers_out

        self.ROM = rom.ROM(d_x, d_f, 
                        scale_factors=sf, 
                        model_path=load_model_path,
                        headers=headers)
        # Define the action space: Bounds, space type, shape

        self.action_space = spaces.Dict({
            'actuator_mode': spaces.MultiDiscrete([3, 3, 3, 3]),  # Example discrete action with 4 options corresponding to 4 modes of the actuators
            'pHP': spaces.Box(low=self.min_rail_pressure, high=self.max_operating_pressure, shape=(1,), dtype=np.float64),
              'pressure_ratio':spaces.Box(low=0, high = 1, shape=(1,), dtype = np.float64),# Example continuous action with shape (2,) for MP and HP
        })

        ## Define the observation space - bounds, space type and shape

        # 4 operating pressures, 4 operating flowrate, 4 delta rpm

        obs_high = np.array([[self.max_operating_pressure] for _ in range(self.num_actuators)] + 
                            [[self.max_operating_flowrate] for _ in range(self.num_actuators)] +
                            [[self.max_delta_rpm] for _ in range(self.num_actuators)])
        # observation space
        self.observation_space = spaces.Box(low=obs_low, high=obs_high, dtype=np.float64) # Observation space including bounds, space type and shape

        # Define exogenous variable
        self.commanded_RPM = np.random.uniform()*self.max_rpm

        self.current_obs = None
        self.episode_simulation_time = 300
        self.simulation_time = 0
        self.dt = 1

    def reset(self):
        # Reset the environment state to the initial state

        #Initial observation

        T, data = utils.load_data('data/data.h5')
        Bulk_P = data['Bulk_P']
        Alt_P = data['Alt_P']
        Vac_P = data['Vac_P']
        Fert_P = data['Fert_P']
        Bulk_Q = data['Bulk_Q']
        Alt_Q = data['Alt_Q']
        Vac_Q = data['Vac_Q']
        Fert_Q = data['Fert_Q']
        Bulk_rpm_delta = data['Bulk_rpm_delta']
        Alt_rpm_delta = data['Alt_rpm_delta']
        Vac_rpm_delta = data['Vac_rpm_delta']
        Fert_rpm_delta = data['Fert_rpm_delta']
        self.init_obs = {'continuous': np.concatenate((Bulk_P[:1, :1], Alt_P[:1, :1], Vac_P[:1, :1], Fert_P[:1, :1],
                                     Bulk_Q[:1, :1], Alt_Q[:1, :1], Vac_Q[:1, :1], Fert_Q[:1, :1],
                                     Bulk_rpm_delta[:1, :1], Alt_rpm_delta[:1, :1], 
                                     Vac_rpm_delta[:1, :1], Fert_rpm_delta[:1, :1]), axis=0)} 
        #TODO: Maybe randomize the index?!
        dt = self.dt # time in seconds

        # Random action

        # Sample high and medium pressure levels at time t = 0
        self.low_pressure = self.min_rail_pressure
        self.high_pressure = np.random.uniform(low=self.low_pressure, high=self.max_operating_pressure)
        self.medium_pressure = max(self.low_pressure, np.random.uniform()*self.high_pressure)
        

        # Sample actuator modes randomly at time t =0 
        self.actuator_modes = np.random.randint(low=1,high=4)*np.ones(self.num_actuators)

        
        # Return the random initial observation from ROM model

        # action = Dict('discrete': NumPy (4, t), 'continuous': NumPy (2, t))
        #                order: Bulk_mode, Alt_mode, Vac_mode, Fert_mode
        #                order: pHP, pMP
        action = {'discrete': self.actuator_modes.reshape(4,1),
                'continuous': np.array([[self.high_pressure],[self.medium_pressure]])}

        # cmd_rpm = Dict('continuous': NumPy (4, t))
        #                order: Bulk_rpm, Alt_rpm, Vac_rpm, Fert_rpm

        # Sample commanded RPM randomly at time t = 0. The commanded RPM is determined by the crops etc.
        self.commanded_RPM = np.random.uniform(low=0, high=self.max_delta_rpm)*np.ones(self.num_actuators)
        cmd_rpm = {'continuous': self.commanded_RPM.reshape(4,1)}

        # obs = Dict('continuous': NumPy (12,))
        #                order: Bulk_P, Alt_P, Vac_P, Fert_P, 
        #                       Bulk_Q, Alt_Q, Vac_Q, Fert_Q,
        #                       Bulk_rpm_delta, Alt_rpm_delta, Vac_rpm_delta, Fert_rpm_delta
        # NOTE: Q values are set to 0 for now

        # Compute random obs from ROM
        obs = self.ROM.call_ROM(act=action, cmd=cmd_rpm, obs=self.init_obs, T=dt)
        self.current_obs = obs['continuous']

        return self.current_obs

    def step(self, action):
        """
        Returns: Given current observation and action, returns the next observation, the reward, done, and additional information
        """

        ## Convert action to the form that ROM can take the input in 

        pHP = action['pHP']
        pMP = pHP*action['pressure_ratio']
        actuator_modes = action['actuator_mode']+1
        action_input = {'discrete': actuator_modes.reshape(4,1),
                'continuous': np.array([pHP,pMP])}
        current_obs = {'continuous':self.current_obs}
        
        ## environment variable

        commanded_RPM = self.environment_variable
        cmd_rpm = {'continuous': commanded_RPM.reshape(4,1)}
        
        ## Compute next observation from action and exogenous variables
        next_obs = self.ROM.call_ROM(act=action_input, cmd=cmd_rpm, obs=current_obs, T=self.dt)

        ## Check if the excess pressure is positive or not
        ## if positive for all actuators, mode in the action is valid.
        ## if negative for any actuator, choose a different mode for that actuator, that would results in a higher pressure drop


        ## Compute the reward
        ### Define excess pressure first 
        pLP = self.low_pressure
        available_pressure = np.array([])
        for i in actuator_modes:
            if i ==1:
                available_pressure_val = pMP-pLP
                available_pressure = np.append(available_pressure, available_pressure_val)
            elif i==2:
                available_pressure_val = pHP-pMP
                available_pressure = np.append(available_pressure, available_pressure_val)
            elif i==3:
                available_pressure_val = pHP-pLP
                available_pressure = np.append(available_pressure, available_pressure_val)
            else:
                raise ValueError("Invalid value for mode. Should be an integer in the list [1,2,3]")

        excess_pressure = available_pressure.reshape(4,1)-next_obs['continuous'][0:4].reshape(4,1)
        reward = -(np.sum(excess_pressure*next_obs['continuous'][4:8].reshape(4,1)) + \
                np.sum(next_obs['continuous'][8:12]))
        
        ## Compute done
        self.simulation_time +=self.dt
        done=False
        if self.simulation_time >= self.episode_simulation_time:
            done = True
        
        # Return the observation, reward, done flag, and additional information

        self.current_obs = next_obs['continuous']
        info = {}  # Additional information (if any)

        return self.current_obs, reward, done, info

    def render(self, mode='human'):
        # Define how to render the environment (optional)
        pass

In [ ]:
# Check if environment works

from controller_env import OptimControllerEnv

env = OptimControllerEnv()
obs = env.reset()

while True:
    action = env.action_space.sample()
    exogenous_variable = env.commanded_RPM
    obs, r, done, _ = env.step(action=action)
    # if r > -2000:
    print(r, action)
    print(obs.flatten())
    
    if done:
        break

In [ ]:
action = env.action_space.sample()
pHP = action['pHP']
pMP = pHP*action['pressure_ratio']
actuator_modes = action['actuator_mode']+1
action_input = {'discrete': actuator_modes.reshape(4,1),
        'continuous': np.array([pHP,pMP])}

# Use Ray-RLlib to solve the custom environment

In [ ]:
import ray
# from ray.rllib.algorithms import ppo
# ray.init()
# algo = ray.rllib.algorithms.ppo.PPO(env=env, config={
#     "env_config": {},  # config to pass to env class
# })

In [ ]:
# ray.rllib.agents.ppo

In [ ]:
from ray.tune.registry import register_env

def env_creator(env_config):
    return OptimControllerEnv()  # return an env instance

register_env("my_env", env_creator)
# algo = ppo.PPO(env="my_env")

In [ ]:
res = tune.run("PPO",
        stop={
        'timesteps_total': 300},
         config = {"env": "my_env",
                   "num_workers": 3,},
         local_dir="./Agent_experiment",
         verbose=0,
         log_to_file=True)